<a href="https://colab.research.google.com/github/InGodWeTrustt/crossword_puzzle/blob/main/%D0%9A%D1%80%D0%BE%D1%81%D1%81%D0%B2%D0%BE%D1%80%D0%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import numpy as np
import random

# Класс для хранения информации о слове
class Word:
    def __init__(self, text, start_x=-1, start_y=-1, direction=None):
        self.text = text
        self.start_x = start_x
        self.start_y = start_y
        self.direction = direction

    def __repr__(self):
        return f"Word('{self.text}', pos=({self.start_x}, {self.start_y}), dir='{self.direction}')"

    # Add __eq__ and __hash__ methods to make Word objects hashable
    def __eq__(self, other):
        if not isinstance(other, Word):
            return False
        return self.text == other.text

    def __hash__(self):
        return hash(self.text)


# Функция для создания пустого поля кроссворда
def create_board(size):
    return np.full((size, size), ' ')

# Функция для проверки, можно ли разместить слово
def can_place_word(board, word_obj, char_index, placed_word_obj):
    word = word_obj.text
    placed_word = placed_word_obj.text
    board_size = board.shape[0]

    # Координаты пересечения на уже размещенном слове
    x_cross, y_cross = -1, -1
    if placed_word_obj.direction == 'horizontal':
        x_cross = placed_word_obj.start_x
        y_cross = placed_word_obj.start_y + char_index
    else: # vertical
        x_cross = placed_word_obj.start_x + char_index
        y_cross = placed_word_obj.start_y

    # Позиция начала нового слова
    new_word_dir = 'vertical' if placed_word_obj.direction == 'horizontal' else 'horizontal'

    # Индекс буквы в новом слове, которая будет пересекаться
    new_char_index = word.find(placed_word[char_index])
    if new_char_index == -1: # Буква не найдена в новом слове
        return False, None, None, None


    if new_word_dir == 'horizontal':
        new_x_start = x_cross
        new_y_start = y_cross - new_char_index
    else: # vertical
        new_x_start = x_cross - new_char_index
        new_y_start = y_cross

    # Проверка границ
    if new_word_dir == 'horizontal':
        if not (0 <= new_y_start and new_y_start + len(word) <= board_size): return False, None, None, None
    else: # vertical
        if not (0 <= new_x_start and new_x_start + len(word) <= board_size): return False, None, None, None

    # Проверка, не пересекается ли слово с другими
    for i, char in enumerate(word):
        x = new_x_start + (i if new_word_dir == 'vertical' else 0)
        y = new_y_start + (i if new_word_dir == 'horizontal' else 0)

        # Проверка ячейки
        if board[x, y] != ' ' and board[x, y] != char:
            return False, None, None, None

        # --- Добавленная проверка на "слипание" ---
        if new_word_dir == 'horizontal':
            # Проверка ячейки слева от слова (если это не пересечение)
            if i == 0 and y > 0 and board[x, y - 1] != ' ': return False, None, None, None
            # Проверка ячейки справа от слова (если это не пересечение)
            if i == len(word) - 1 and y < board_size - 1 and board[x, y + 1] != ' ': return False, None, None, None
            # Проверка ячеек сверху и снизу (кроме точки пересечения)
            # Учитываем, что в точке пересечения могут быть уже буквы других слов
            if not (x == x_cross and y == y_cross): # Если это не точка пересечения
                if x > 0 and board[x-1, y] != ' ': return False, None, None, None
                if x < board_size-1 and board[x+1, y] != ' ': return False, None, None, None
        else: # vertical
            # Проверка ячейки сверху от слова (если это не пересечение)
            if i == 0 and x > 0 and board[x - 1, y] != ' ': return False, None, None, None
            # Проверка ячейки снизу от слова (если это не пересечение)
            if i == len(word) - 1 and x < board_size - 1 and board[x + 1, y] != ' ': return False, None, None, None
            # Проверка ячеек слева и справа (кроме точки пересечения)
            # Учитываем, что в точке пересечения могут быть уже буквы других слов
            if not (x == x_cross and y == y_cross): # Если это не точка пересечения
                if y > 0 and board[x, y-1] != ' ': return False, None, None, None
                if y < board_size-1 and board[x, y+1] != ' ': return False, None, None, None


    return True, new_x_start, new_y_start, new_word_dir

# Оценка размещения слова
def evaluate_placement(board, word_obj, all_words):
    score = 0
    # Проверка на совпавшие буквы (уже учтено в can_place_word, но можно добавить балл)

    # Эвристика: оценка новых потенциальных пересечений
    for i, char in enumerate(word_obj.text):
        x = word_obj.start_x + (i if word_obj.direction == 'vertical' else 0)
        y = word_obj.start_y + (i if word_obj.direction == 'horizontal' else 0)

        potential_matches = [w for w in all_words if char in w]

        temp_board = board.copy()
        place_word(temp_board, word_obj)

        if word_obj.direction == 'horizontal':
            # Ищем, сколько вертикальных слов могут пересечься
            if x > 0 and temp_board[x-1, y] == ' ' and x < temp_board.shape[0] - 1 and temp_board[x+1, y] == ' ':
                for p_word in potential_matches:
                    if p_word != word_obj.text and len(p_word) > 1:
                        score += 1
        else: # vertical
            if y > 0 and temp_board[x, y-1] == ' ' and y < temp_board.shape[1] - 1 and temp_board[x, y+1] == ' ':
                for p_word in potential_matches:
                    if p_word != word_obj.text and len(p_word) > 1:
                        score += 1

    return score

# Функция для размещения слова на доске
def place_word(board, word_obj):
    word = word_obj.text
    for i, letter in enumerate(word):
        x = word_obj.start_x + (i if word_obj.direction == 'vertical' else 0)
        y = word_obj.start_y + (i if word_obj.direction == 'horizontal' else 0)
        board[x, y] = letter

# Основная функция для генерации кроссворда
def generate_crossword(words, board_size=20):
    board = create_board(board_size)
    words.sort(key=len, reverse=True)
    placed_words_objs = []
    placed_word_texts = set() # Set to keep track of placed word texts to avoid duplicates
    used_positions = set() # Set to store coordinates of placed letters

    # Try placing the longest word in the center horizontally first, then vertically if horizontal fails
    center_x, center_y = board_size // 2, board_size // 2
    first_word_text = words[0]

    # Try horizontal placement
    first_word_obj_h = Word(first_word_text, start_x=center_x, start_y=center_y - len(first_word_text) // 2, direction='horizontal')
    if 0 <= first_word_obj_h.start_y and first_word_obj_h.start_y + len(first_word_text) <= board_size:
        place_word(board, first_word_obj_h)
        placed_words_objs.append(first_word_obj_h)
        placed_word_texts.add(first_word_text)
        for i in range(len(first_word_text)):
             used_positions.add((first_word_obj_h.start_x, first_word_obj_h.start_y + i))
        print(f"Разместили первое слово: {first_word_obj_h.text}")
    else:
        # Try vertical placement if horizontal failed or didn't fit well
        board = create_board(board_size) # Reset board
        first_word_obj_v = Word(first_word_text, start_x=center_x - len(first_word_text) // 2, start_y=center_y, direction='vertical')
        if 0 <= first_word_obj_v.start_x and first_word_obj_v.start_x + len(first_word_text) <= board_size:
             place_word(board, first_word_obj_v)
             placed_words_objs.append(first_word_obj_v)
             placed_word_texts.add(first_word_text)
             for i in range(len(first_word_text)):
                 used_positions.add((first_word_obj_v.start_x + i, first_word_obj_v.start_y))
             print(f"Разместили первое слово (вертикально): {first_word_obj_v.text}")
        else:
            print(f"Не удалось разместить первое слово в центре: {first_word_text}")
            return board, placed_words_objs, used_positions # Cannot place the first word, return empty board and positions


    for i in range(1, len(words)):
        current_word = words[i]
        if current_word in placed_word_texts: # Skip if word is already placed
            continue
        best_placement = None
        max_score = -1


        random.shuffle(placed_words_objs) # Shuffle placed words to add some randomness to placement

        for placed_word_obj in placed_words_objs:
            # Shuffle characters to add some randomness to intersection points
            char_indices = list(range(len(placed_word_obj.text)))
            random.shuffle(char_indices)

            for char_index in char_indices:
                char_in_placed = placed_word_obj.text[char_index]
                if char_in_placed in current_word:
                    can_place, x, y, direction = can_place_word(board, Word(current_word), char_index, placed_word_obj)
                    if can_place:
                        temp_word_obj = Word(current_word, x, y, direction)
                        score = evaluate_placement(board, temp_word_obj, words)
                        if score > max_score:
                            max_score = score
                            best_placement = (x, y, direction)

        if best_placement:
            x, y, direction = best_placement
            new_word_obj = Word(current_word, x, y, direction)
            place_word(board, new_word_obj)
            placed_words_objs.append(new_word_obj)
            placed_word_texts.add(current_word) # Add word text to the set
            # Add coordinates to used_positions
            for i in range(len(current_word)):
                if direction == 'horizontal':
                    used_positions.add((x, y + i))
                else: # vertical
                    used_positions.add((x + i, y))
            # print(f"Разместили слово: {new_word_obj.text} с оценкой: {max_score}") # Commented out this line
        else:
            # print(f"Не удалось разместить слово: {current_word}") # Commented out this line
            pass # Do nothing if word cannot be placed


    return board, placed_words_objs, used_positions

def print_board(board):
    for row in board:
        print(' '.join(row))

if __name__ == "__main__":
    words_list = []
    file_path = '/content/russian (6).txt'
    board_size=15

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                word = line.strip().upper() # Read word, remove whitespace, convert to uppercase
                # Filter words: not empty, only alphabetic characters, and length <= 15
                if word and word.isalpha() and len(word) <= 9: # Changed max length to 9
                    words_list.append(word)
    except FileNotFoundError:
        print(f"Файл не найден: {file_path}")
        words_list = ["ПРИМЕР", "СЛОВО", "ТЕСТ"] # Use a default list if file not found
    except Exception as e:
        print(f"Произошла ошибка при чтении файла: {e}")
        words_list = ["ПРИМЕР", "СЛОВО", "ТЕСТ"] # Use a default list in case of other errors


    random.shuffle(words_list) # Shuffle the word list


    # Измените размер доски на больший, например 30
    crossword_board, placed_words, used_positions = generate_crossword(words_list, board_size=15) # Changed board size to 15

    closest_word = None
    # Calculate the average length of placed words
    if placed_words:
        total_length = sum(len(word.text) for word in placed_words)
        average_length = total_length / len(placed_words)
        print(f"\nСредняя длина размещенных слов: {average_length:.2f}")

        # Find the word closest to the average length

        min_diff = float('inf')

        for word_obj in placed_words:
            diff = abs(len(word_obj.text) - average_length)
            if diff < min_diff:
                min_diff = diff
                closest_word = word_obj

        if closest_word:
            print(f"Слово, ближайшее по длине к среднему: {closest_word.text} (длина: {len(closest_word.text)})")
        else:
            print("Не удалось найти слово, ближайшее по длине к среднему.")
    else:
        print("\nНет размещенных слов для расчета средней длины.")


    board_with_one_word = create_board(board_size)
    if closest_word:
        place_word(board_with_one_word, closest_word)


    # Формируем кросворды
    crossword_data_list = [board_with_one_word, crossword_board]
    export_to_docx(crossword_data_list,placed_words, closest_word, used_positions)

    # print(closest_word)
    # print(board_with_one_word)
    """
    print("\n--- Сгенерированный кроссворд ---")
    print_board(crossword_board)

    print("\n--- Список размещенных слов ---")
    for word in placed_words:
        print(word)

    print("\n--- Занятые позиции ---")
    print(used_positions)

    print(placed_words)
    """

Разместили первое слово: ГОЛОСОВОЕ

Средняя длина размещенных слов: 5.48
Слово, ближайшее по длине к среднему: СКЛОН (длина: 5)
[Word('СКЛОН', pos=(8, 14), dir='vertical'), Word('РУМЯНЫЙ', pos=(12, 0), dir='horizontal'), Word('БОНДАРЬ', pos=(2, 0), dir='horizontal'), Word('ТУШ', pos=(0, 10), dir='vertical'), Word('ДВИГАТЬСЯ', pos=(4, 3), dir='vertical'), Word('МАРШРУТ', pos=(7, 1), dir='vertical'), Word('СКЛЕРОТИК', pos=(4, 11), dir='vertical'), Word('ТАР', pos=(12, 12), dir='vertical'), Word('ВОДОСПУСК', pos=(3, 7), dir='vertical'), Word('ОКТАН', pos=(12, 10), dir='horizontal'), Word('ЩИ', pos=(6, 2), dir='horizontal'), Word('АЗУ', pos=(1, 8), dir='horizontal'), Word('ГОЛОСОВОЕ', pos=(7, 3), dir='horizontal'), Word('ЧАРОДЕЙ', pos=(0, 0), dir='horizontal'), Word('ЧУБАРЫЙ', pos=(0, 0), dir='vertical'), Word('СОК', pos=(14, 4), dir='horizontal'), Word('ШКИВ', pos=(2, 10), dir='horizontal'), Word('НОС', pos=(12, 4), dir='vertical'), Word('ТОП', pos=(0, 10), dir='horizontal'), Word('СЫР', 

In [71]:
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.table import WD_ALIGN_VERTICAL
from docx.enum.section import WD_ORIENT
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
from docx.enum.table import WD_ROW_HEIGHT_RULE

def export_to_docx(crossword_data_list, placed_words, closest_word, used_positions):
        """Экспорт в Word с правильными границами"""
        doc = Document()

        print(placed_words)
        section = doc.sections[0]
        # Настройка альбомной ориентацииsection = doc.sections[0]
        section.orientation = WD_ORIENT.LANDSCAPE
        # Меняем местами ширину и высоту для альбомной ориентации
        new_width, new_height = section.page_height, section.page_width
        section.page_width = new_width
        section.page_height = new_height

        # Уменьшаем отступы
        section.left_margin = Inches(0.5)
        section.right_margin = Inches(0.5)
        section.top_margin = Inches(0.5)
        section.bottom_margin = Inches(0.5)

        # Группируем филворды по парам для размещения на странице
        for page_num in range(0, len(crossword_data_list), 2):
            if page_num > 0:
                doc.add_page_break()

            # Получаем до 2 филвордов для текущей страницы
            crossword_on_page = crossword_data_list[page_num:page_num + 2]
            add_crossword_to_page(doc, crossword_on_page, page_num // 2 + 1, placed_words, used_positions, closest_word)

        doc.save('anti_crossword_with_structure.docx')

def add_crossword_to_page(doc, crosswords, page_number, words_list, used_positions, closest_word):
    if len(crosswords) == 2:
        # Два филворда в горизонтальном ряду
        add_two_crosswords_horizontal(doc, crosswords,words_list, used_positions , closest_word)
    else:
        # Один филворд (последняя страница)
        add_single_crossword(doc, crosswords[0], words_list, used_positions , closest_word)

def add_two_crosswords_horizontal(doc, crosswords, words_list, used_positions, closest_word):
    """Добавляет два филворда горизонтально"""

    # print(crosswords)
    # Создаем таблицу для размещения двух филвордов
    main_table = doc.add_table(rows=1, cols=3)
    main_table.autofit = False

    #для каждой ячейки таблицы устанавливаем

    # Настраиваем ширину колонок через Inches
    widths = [Inches(5.05), Inches(0.3),Inches(5.05)]  # 12 см, 2 см, 12 см (в дюймах)
    header_cells = main_table.rows[0].cells  # Получаем ячейки первой строки
    for j in range(len(widths)):  # Проходим по всем колонкам
        header_cells[j].width = widths[j]  # Устанавливаем ширину для каждой ячейки

    left_cell = main_table.cell(0, 0)
    right_cell = main_table.cell(0, 2)

    for cell in [left_cell, right_cell]:
        for par in cell.paragraphs:
            p = par._element
            p.getparent().remove(p)

    # Добавляем первый филворд
    add_crossword_to_cell(left_cell, crosswords[0], words_list, used_positions, closest_word)

    # Добавляем второй филворд
    add_crossword_to_cell(right_cell, crosswords[1], words_list, used_positions, closest_word)

    # Убираем границы главной таблицы
    remove_table_borders(main_table)


def add_single_crossword(doc, filword_data, words_list, used_positions , closest_word):
    """Добавляет один филворд по центру"""
    # Добавляем сетку
    add_grid_table(doc, filword_data)

    # Добавляем список слов
    add_words_list(doc, words_list)

def add_crossword_to_cell(cell, filword_data, words_list, used_positions, closest_word):
    """Добавляет филворд в ячейку таблицы"""


    # Создаем таблицу для сетки филворда
    grid = filword_data
    size = len(grid)

    grid_table = cell.add_table(rows=size, cols=size)
    # Устанавливаем стиль таблицы с сеткой
    grid_table.style = 'Table Grid'
    grid_table.autofit = False

    # Устанавливаем таблицу чуть меньше ширины ячейки
    grid_table.preferred_width = Inches(5)  # Чуть меньше ширины колонки родительской таблицы (12 см)
    grid_table.alignment = WD_ALIGN_PARAGRAPH.CENTER

    # Рассчитываем размер ячейки исходя из ширины таблицы
    cell_width = Inches(5 / size)  # Делим ширину на количество ячеек

    # Находим пустые клетки, окружённые буквами
    black_cells = find_cells_to_fill(used_positions, size, size)


    # Настраиваем размеры ячеек и заполняем данными
    for i in range(size):
        for j in range(size):
            cell_obj = grid_table.cell(i, j)
            cell_obj.width = cell_width

            if (i, j) in used_positions:
                # Ячейка входит в какое-то слово → рисуем границы
                set_cell_border(cell_obj, top='single', left='single', bottom='single', right='single')

            else:
                # Ячейка НЕ входит в слово → убираем все границы
                set_cell_border(cell_obj, top='nil', left='nil', bottom='nil', right='nil')

                if (i,j) in black_cells:
                    set_cell_background_color(cell_obj,'000000') # Черный цвет

            # Устанавливаем поля ячейки
            set_cell_margins(cell_obj, top=0, start=0, bottom=0, end=0)

            # Добавляем букву
            p = cell_obj.paragraphs[0]
            p.alignment = WD_ALIGN_PARAGRAPH.CENTER

            # Выравниваем по вертикали
            cell_obj.vertical_alignment = WD_ALIGN_VERTICAL.CENTER

            run = p.add_run(grid[i][j])
            run.font.size = Pt(20)
            run.font.name = 'Times New Roman'
            run.bold = True


    for row in grid_table.rows:
        row.height = Pt(20)
        row.height_rule = WD_ROW_HEIGHT_RULE.AT_LEAST

    # Группируем слова в колонки, сортируем по длине
    words = [word.text for word in sorted(words_list, key=lambda x: len(x.text))]

    # Рассчитываем количество колонок по ширине филворда (15 ячеек * 0.2 дюйма = 3 дюйма, делим на ширину слова)
    columns = max(3, min(6, len(words) // 8))  # от 3 до 6 колонок

    add_words_columns(cell, words, columns=columns, closest_word=closest_word)

def add_words_list(doc, words):
    """Добавляет список слов в несколько колонок"""

    # Убираем заголовок "Слова для поиска"
    words_list = [word.text for word in sorted(words, key=lambda x: len(x.text))]
    add_words_columns_to_doc(doc, words_list, columns=4)

def add_words_columns(parent_cell, words, columns=3, closest_word=None):
    """Добавляет слова в колонки внутри ячейки"""

    if not words:
        return

    # Создаем таблицу для слов
    rows_needed = (len(words) + columns - 1) // columns
    words_table = parent_cell.add_table(rows=rows_needed, cols=columns)
    words_table.autofit = False
    words_table.style = "Table Grid"

    # Заполняем таблицу словами
    for idx, word in enumerate(words):
        col = idx // rows_needed
        row = idx % rows_needed

        cell = words_table.cell(row, col)
        p = cell.paragraphs[0]
        # установить интервал ячейки в 12 пунктов для первого ряд
        if row == 0:
            paragraph_format = p.paragraph_format
            paragraph_format.space_before = Pt(12)

        run = p.add_run(f"{word.upper()}")
        run.font.size = Pt(9) if len(word) > 10 else Pt(12)
        run.bold = True if len(word) > 10 else False

        # Зачеркиваем показанное первое слово
        if closest_word is not None and word.upper() == closest_word.text.upper():
            run.font.strike = True

        run.font.name = 'Times New Roman'

    # Убираем границы таблицы со словами
    remove_table_borders(words_table)

def add_words_columns_to_doc(doc, words, columns=4):
    """Добавляет слова в колонки в документ"""
    if not words:
        return

    rows_needed = (len(words) + columns - 1) // columns
    words_table = doc.add_table(rows=rows_needed, cols=columns)
    words_table.alignment = WD_ALIGN_PARAGRAPH.CENTER

    for idx, word in enumerate(words):
        col = idx // rows_needed
        row = idx % rows_needed

        cell = words_table.cell(row, col)
        p = cell.paragraphs[0]
        p.alignment = WD_ALIGN_PARAGRAPH.LEFT
        run = p.add_run(f"{word.upper()}")
        run.font.size = Pt(12)
        run.font.name = 'Times New Roman'

    remove_table_borders(words_table)

def remove_table_borders(table):
    """Убирает границы таблицы"""

    tbl = table._tbl
    for row in tbl.tr_lst:
        for cell in row.tc_lst:
            tcPr = cell.get_or_add_tcPr()
            tcBorders = tcPr.first_child_found_in("w:tcBorders")

            if tcBorders is None:
                tcBorders = OxmlElement('w:tcBorders')
                tcPr.append(tcBorders)

            # Убираем границы
            for border_name in ['top', 'left', 'bottom', 'right']:
                border = OxmlElement(f'w:{border_name}')
                border.set(qn('w:val'), 'nil')
                tcBorders.append(border)

def set_cell_margins(cell, top=50, start=50, bottom=50, end=50):
    """
Устанавливает отступы ячейки (в Twips: 1/1440 дюйма, 1 pt = 20 Twips)
5 pt ≈ 100 Twips
    """
    tc = cell._tc
    tcPr = tc.get_or_add_tcPr()
    tcMar = OxmlElement('w:tcMar')
    for margin, value in [('w:top', top), ('w:start', start),
                        ('w:bottom', bottom), ('w:end', end)]:
        node = OxmlElement(margin)
        node.set(qn('w:w'), str(value))
        node.set(qn('w:type'), 'dxa')
        tcMar.append(node)

    tcPr.append(tcMar)


def set_cell_border(cell, top=None, left=None, bottom=None, right=None):
    tc = cell._tc
    tcPr = tc.get_or_add_tcPr()
    tcBorders = tcPr.first_child_found_in("w:tcBorders")
    if tcBorders is None:
        tcBorders = OxmlElement('w:tcBorders')
        tcPr.append(tcBorders)

    for border_name, border_val in [('top', top), ('left', left), ('bottom', bottom), ('right', right)]:
        if border_val is not None:
            tag = f'w:{border_name}'
            element = tcBorders.find(qn(tag))
            if element is None:
                element = OxmlElement(tag)
                tcBorders.append(element)
            element.set(qn('w:val'), border_val)
            element.set(qn('w:sz'), '4')
            element.set(qn('w:space'), '0')
            element.set(qn('w:color'), 'auto')

def remove_table_borders(table):
    """Убирает границы таблицы"""

    tbl = table._tbl
    for row in tbl.tr_lst:
        for cell in row.tc_lst:
            tcPr = cell.get_or_add_tcPr()
            tcBorders = tcPr.first_child_found_in("w:tcBorders")

            if tcBorders is None:
                tcBorders = OxmlElement('w:tcBorders')
                tcPr.append(tcBorders)

            # Убираем границы
            for border_name in ['top', 'left', 'bottom', 'right']:
                border = OxmlElement(f'w:{border_name}')
                border.set(qn('w:val'), 'nil')
                tcBorders.append(border)

def add_grid_table(doc, grid):
        """Добавляет таблицу с сеткой филворда"""

        size = len(grid)
        table = doc.add_table(rows=size, cols=size)
        table.alignment = WD_ALIGN_PARAGRAPH.CENTER
        table.autofit = False

        # Устанавливаем таблицу чуть меньше полной ширины страницы
        table.preferred_width = Inches(7.5)  # Чуть меньше ширины страницы A4

        # Рассчитываем размер ячейки
        cell_width = Inches(7.5 / size)

        for i in range(size):
            for j in range(size):
                cell = table.cell(i, j)
                cell.width = cell_width
                p = cell.paragraphs[0]
                p.alignment = WD_ALIGN_PARAGRAPH.CENTER
                # Выравниваем по вертикали
                cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER
                run = p.add_run(grid[i][j])
                run.font.size = Pt(20)
                run.font.name = 'Times New Roman'
                run.bold = True

        # Устанавливаем стиль таблицы с сеткой
        table.style = 'Table Grid'

def find_cells_to_fill(letters_set, rows, cols):
    """
            Возвращает множество координат (row, col), которые нужно закрасить.

                    :param letters_set: множество кортежей (r, c) — где находятся буквы
                        :param rows: общее количество строк в сетке
                            :param cols: общее количество столбцов
                                :return: множество (row, col) для закрашивания
                                    """

    black_cells = set()

    # Соседи: вверх, вниз, влево, вправо
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    # Проходим только по внутренним ячейкам
    for r in range(1, rows - 1):
        for c in range(1, cols - 1):
            # Если ячейка НЕ пустая (в ней буква) — пропускаем
            if (r, c) in letters_set:
                continue

            surrounded = True
            for dr, dc in directions:
                nr, nc = r + dr, c + dc
                if (nr, nc) not in letters_set:
                    surrounded = False
                    break
            if surrounded:
                black_cells.add((r, c))
    return black_cells

# Изменить цвет ячейки в python-docx from docx import Document
from docx.oxml import OxmlElement
from docx.oxml.ns import qn

def set_cell_background_color(cell, color):
      """
          Устанавливает цвет фона для ячейки.
          :param cell: Ячейка для форматирования
          :param color: Цвет в формате hex-строки (без символа #)
          """
      # Получаем элемент свойств ячейки
      tcPr = cell._element.get_or_add_tcPr()

      # Создаем элемент затенения
      shd = OxmlElement('w:shd')
      shd.set(qn('w:fill'),color)

      # Добавляем элемент к свойствам ячейки
      tcPr.append(shd)

      # Пример использования
      doc = Document()

      # Создаем таблицу
      table = doc.add_table(rows=2,cols=2)

      # Заполняем ячейки текстом
      table.cell(0,0).text = "Ячейка 1"
      table.cell(0, 1).text = "Ячейка 2"
      table.cell(1, 0).text = "Ячейка 3"
      table.cell(1, 1).text = "Ячейка 4"

      # Закрашиваем ячейку в черный цвет
      # set_cell_background_color(table.cell(0,1),'000000')# Черный цвет

In [ ]:
!pip install python-docx

In [ ]:
from google.colab import files

files.download('anti_crossword_with_structure.docx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>